In [1]:
# -------------------------------
# CIFAR-100 Multi-seed, multi-class evaluation with .npz export + timing
# -------------------------------

import os
import time
import numpy as np
import torch
import torch.nn as nn
from lib.cifar100_utils import (
    build_cifar_resnet, build_wideresnet, get_dataloaders, load_model, evaluate_model
)

# -------------------------------
# Configuration
# -------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 512
NUM_CLASSES = 100
MODEL_DIR = "./../models/cifar100/"
DATASET_DIR = "./../data/"
RESULTS_DIR = "./../results/cifar100/"
os.makedirs(RESULTS_DIR, exist_ok=True)

SEEDS = [42, 602, 311, 637, 800, 543, 969, 122, 336, 93]
REMOVE_CLASSES = [
    None, 4, 7, 6, 11, 14, 15, 19, 24, 23, 25, 29, 30, 
    31, 36, 35, 39, 44, 41, 49, 48, 51, 53, 59, 57, 60, 61, 
    68, 67, 72, 71, 76, 77, 80, 84, 88, 85, 92, 94, 99, 97] # CIFAR-100

EPOCHS = 50

MODEL_TYPE = "resnet18"  # "resnet18" or "wideresnet"

# CSV for logging time
TIME_CSV = "./../analytics/CIFAR100/timer/cifar100_eval_times.csv"
os.makedirs(os.path.dirname(TIME_CSV), exist_ok=True)
import csv
with open(TIME_CSV, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["seed", "removed_class", "time_seconds"])

# -------------------------------
# Run evaluation and save results
# -------------------------------
for seed in SEEDS:
    print(f"\n=== Seed {seed} ===")
    seed_start = time.time()

    for remove_class in REMOVE_CLASSES:
        start_time = time.time()
        print(f"Evaluating seed={seed}, remove_class={remove_class}...")

        # Prepare test data
        _, testloader = get_dataloaders(
            batch_size=BATCH_SIZE,
            remove_class=remove_class,
            dataset_dir=DATASET_DIR
        )

        # Build model path
        model_name = f"cifar100_{MODEL_TYPE}_s{seed}_e{EPOCHS}"
        if remove_class is not None:
            model_name += f"_r{remove_class}"
        model_path = os.path.join(MODEL_DIR, model_name + ".pth")

        # Build result path
        result_path = os.path.join(RESULTS_DIR, model_name + ".npz")

        # Skip if already evaluated
        #if os.path.exists(result_path):
            #print(f"Already exists: {result_path}")
            #continue

        # Build model
        if MODEL_TYPE == "resnet18":
            model = build_cifar_resnet(num_classes=NUM_CLASSES, device=DEVICE)
        elif MODEL_TYPE == "wideresnet":
            model = build_wideresnet(num_classes=NUM_CLASSES, device=DEVICE)
        else:
            raise ValueError(f"Unknown model type: {MODEL_TYPE}")

        load_model(model, model_path, device=DEVICE)

        preds, targets, confs = evaluate_model(
            model, testloader, nn.CrossEntropyLoss(), device=DEVICE
        )

        # Save results to npz
        np.savez_compressed(result_path, preds=preds, targets=targets, confs=confs)

        elapsed = time.time() - start_time
        print(f"Saved: {result_path} | Inference time: {elapsed:.3f}s")

        # Log time to CSV
        with open(TIME_CSV, mode="a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([seed, remove_class, elapsed])

    seed_elapsed = time.time() - seed_start
    print(f"✅ Completed seed {seed} in {seed_elapsed:.2f} s")

print("\nAll models evaluated and times saved successfully.")



=== Seed 42 ===
Evaluating seed=42, remove_class=None...
Model loaded from ./../models/cifar100/cifar100_resnet18_s42_e50.pth
Test Loss: 1.085 | Test Acc: 73.40%
Saved: ./../results/cifar100/cifar100_resnet18_s42_e50.npz | Inference time: 4.493s
Evaluating seed=42, remove_class=4...
Model loaded from ./../models/cifar100/cifar100_resnet18_s42_e50_r4.pth
Test Loss: 1.169 | Test Acc: 73.43%
Saved: ./../results/cifar100/cifar100_resnet18_s42_e50_r4.npz | Inference time: 3.372s
Evaluating seed=42, remove_class=7...
Model loaded from ./../models/cifar100/cifar100_resnet18_s42_e50_r7.pth
Test Loss: 1.207 | Test Acc: 73.04%
Saved: ./../results/cifar100/cifar100_resnet18_s42_e50_r7.npz | Inference time: 3.308s
Evaluating seed=42, remove_class=6...
Model loaded from ./../models/cifar100/cifar100_resnet18_s42_e50_r6.pth
Test Loss: 1.198 | Test Acc: 72.69%
Saved: ./../results/cifar100/cifar100_resnet18_s42_e50_r6.npz | Inference time: 3.415s
Evaluating seed=42, remove_class=11...
Model loaded fr